In [1]:
#import dependencies

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException, TimeoutException

In [2]:
import time
import requests
from bs4 import BeautifulSoup
import re
from dotenv import load_dotenv
import json
import os
from urllib.parse import urlparse 
import csv

load_dotenv()

True

In [3]:
#setup chromedriver
driver = webdriver.Chrome()

# open webpage
driver.get("https://www.instagram.com/")

In [4]:
# target username
username = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "input[name='username']")))
password = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "input[name='password']")))


#enter username and password
username.clear()
username.send_keys(os.getenv('INSTAGRAM_USERNAME'))
password.clear()
password.send_keys(os.getenv('INSTAGRAM_PASSWORD'))

#target the login button and click it
button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button[type='submit']")))

In [6]:
# wait a few seconds for the search button to be clickable
search_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "svg[aria-label='Search']")))

# wait until button is clickable
search_button.click()

In [7]:
# target the search input field
searchbox = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//input[@placeholder='Search']")))
searchbox.clear()

# search for the @handle orkeyword
keyword = "bank.repossessedcars"
searchbox.send_keys(keyword)

In [8]:
# check if the keyword starts with @
if keyword.startswith("@"):
    # remove @ symbol
    keyword = keyword[1:]

# find the first element with specified xpath that matches the keyword
time.sleep(5)
first_result = driver.find_element(By.XPATH, f'//span[text()="{keyword}"]')
first_result.click()

In [9]:
# get the initial page height
initial_height = driver.execute_script("return document.body.scrollHeight")
# create a list to store htmls
soups = []
# scroll loop
while True:
    # scroll down to the bottom of the page
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    # wait a moment to allow new content to load 
    time.sleep(5)
    # parse the html
    html = driver.page_source
    # create beautifulsoup object from scrapped html
    soups.append(BeautifulSoup(html, 'html.parser'))

    # get current page height
    current_height = driver.execute_script("return document.body.scrollHeight")
    if current_height == initial_height:
        break# exit the loop when you cant scroll down any further
    # update the initial height for the next iteration 
    initial_height = current_height

In [10]:
# list to store the post image urls
post_urls = []

# loop through the soup elements 
for soup in soups:

    # find all image elements that match the specific class in soup 
    elements = soup.find_all('a', class_='x1i10hfl xjbqb8w x1ejq31n xd10rxx x1sy0etr x17r0tee x972fbf xcfux6l x1qhh985 xm0m39n x9f619 x1ypdohk xt0psk2 xe8uvvx xdj266r x11i5rnm xat24cr x1mh8g0r xexx8yu x4uap5 x18d9i69 xkhd6sd x16tdsg8 x1hl2dhg xggy1nq x1a2a7pz _a6hd')

    # extract the href attributes and filter urls that start with "/p/" or "/ree/"
    post_urls.extend([element['href'] for element in elements if element['href'].startswith(("/bank.repossessedcars/reel/", "/bank.repossessedcars/p/"))])

# convert list to a set to remove duplicates 
unique_post_urls = list(set(post_urls))


In [12]:
unique_post_urls

['/bank.repossessedcars/p/DCTfJ9fIinp/',
 '/bank.repossessedcars/p/DC6tuV2o6Dc/',
 '/bank.repossessedcars/p/DEje1z1tUzV/',
 '/bank.repossessedcars/p/DC3-o2NIXMX/',
 '/bank.repossessedcars/p/DC6Ks5soHso/',
 '/bank.repossessedcars/p/DCTIK7YIHkL/',
 '/bank.repossessedcars/p/DCn5UkVoQ6G/',
 '/bank.repossessedcars/p/DDyqqf7tOU5/',
 '/bank.repossessedcars/p/DEeO9r8tZZ1/',
 '/bank.repossessedcars/p/DEXZzFKNsZE/',
 '/bank.repossessedcars/p/DB3iFERoZge/',
 '/bank.repossessedcars/p/DDWS8qoqxup/',
 '/bank.repossessedcars/p/DC4HCgQIUgM/',
 '/bank.repossessedcars/p/DDy489cNmIZ/',
 '/bank.repossessedcars/p/DCOjzEgoLqv/',
 '/bank.repossessedcars/p/DB-5r56oOar/',
 '/bank.repossessedcars/p/DEXLMneNuOy/',
 '/bank.repossessedcars/p/DEeucT6tYcq/',
 '/bank.repossessedcars/p/DC5_fjgIB02/',
 '/bank.repossessedcars/p/DCPBC9poSoX/',
 '/bank.repossessedcars/p/DDXFuaPq9D2/',
 '/bank.repossessedcars/reel/DA1lfOboBtE/',
 '/bank.repossessedcars/p/DDWO4Hnq8EC/',
 '/bank.repossessedcars/p/DEXtWont2hC/',
 '/bank.repos

In [17]:
# # create a list to store the json for each post 
# json_list = []

# # define the query parameters 
# query_parameters = "__a=1&__d=dis"

# # go through the urls 
# for url in unique_post_urls:

#     # error handling 
#     try:

#         # get the current url of the page 
#         current_url = driver.current_url
#         # append the query parameters to the current url
#         modified_url = "https://www.instagram.com/"  + url + "?" + query_parameters

#         # get url 
#         driver.get(modified_url)

#         # wait for a moment to allow new content to load(adjust as needed)
#         time.sleep(1)

#         # find the <pre> tag containing json data 
#         WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "//pre")))
#         pre_tag = driver.find_element(By.XPATH, '//pre')

#         # extract the json data from the pre tag 
#         json_script = pre_tag.text

#         # parse the json data 
#         json_parsed = json.loads(json_script)

#         # add json to the list 
#         json_list.append(json_parsed)

#     # error handling 
#     except (NoSuchElementException, TimeoutException, json.JSONDecodeError) as e:
#     print(f"Error processing URL {url}: {e}")

In [15]:
import httpx
import json
from typing import Dict
from urllib.parse import quote
# # create a list to store the json for each post 
json_list = []

# # go through the urls 
INSTAGRAM_DOCUMENT_ID = "8845758582119845" # contst id for post documents instagram.com

for url in unique_post_urls:
    """Scrape single Instagram post data"""
    if "/p/" in url:
        shortcode = url.split('/p/')[-1].split('/')[0]
    else:
        print(f"Invalid URL format, skipping: {url}")
        continue  # Skip this iteration if the URL format is incorrect

    variables = quote(json.dumps({
        'shortcode':shortcode,
        'fetch_tagged_user_count':None,
        'hoisted_comment_id':None,
        'hoisted_reply_id':None
    }, separators=(',', ':')))
    body = f"variables={variables}&doc_id={INSTAGRAM_DOCUMENT_ID}"
    url = "https://www.instagram.com/graphql/query"

    # wait for a moment to allow new content to load(adjust as needed)
    time.sleep(1)

    result = httpx.post(
        url=url,
        headers={"content-type": "application/x-www-form-urlencoded"},
        data=body
    )

    # parse the json data 
    json_parsed = json.loads(result.content)

    # add json to the list 
    json_list.append(json_parsed)
    

Invalid URL format, skipping: /bank.repossessedcars/reel/DA1lfOboBtE/
Invalid URL format, skipping: /bank.repossessedcars/reel/DC7JzK4o61C/
Invalid URL format, skipping: /bank.repossessedcars/reel/DCqwP2jI1i3/
Invalid URL format, skipping: /bank.repossessedcars/reel/DCZIaCLosYn/
Invalid URL format, skipping: /bank.repossessedcars/reel/C8tf0sBIsi1/
Invalid URL format, skipping: /bank.repossessedcars/reel/DBxtd6moRVJ/
Invalid URL format, skipping: /bank.repossessedcars/reel/C-CtYh6IhR9/
Invalid URL format, skipping: /bank.repossessedcars/reel/DBv-8d2oZxJ/
Invalid URL format, skipping: /bank.repossessedcars/reel/DC9KIKWIzqs/
Invalid URL format, skipping: /bank.repossessedcars/reel/C8_zpsUo0r-/
Invalid URL format, skipping: /bank.repossessedcars/reel/DBA3yvKIpE_/
Invalid URL format, skipping: /bank.repossessedcars/reel/DCWKDMoIF8n/
Invalid URL format, skipping: /bank.repossessedcars/reel/C8BeH2VIpHl/
Invalid URL format, skipping: /bank.repossessedcars/reel/DBQHXP1oifH/


In [16]:
json_list

[{'data': {'xdt_shortcode_media': {'__typename': 'XDTGraphSidecar',
    '__isXDTGraphMediaInterface': 'XDTGraphSidecar',
    'id': '3500278359334726121',
    'shortcode': 'DCTfJ9fIinp',
    'thumbnail_src': 'https://instagram.fnbo16-1.fna.fbcdn.net/v/t51.29350-15/466663621_3746129452294615_7696885102146473236_n.webp?stp=c100.0.949.949a_dst-jpg_e35_s640x640_sh0.08_tt6&_nc_ht=instagram.fnbo16-1.fna.fbcdn.net&_nc_cat=107&_nc_ohc=Gq1MQNeg7LQQ7kNvgHiOPK0&_nc_gid=8246ebf5bdd44667bf18f51a8c4f3ff9&edm=ANTKIIoBAAAA&ccb=7-5&oh=00_AYB49Lt049C1zhQ5hdwN5aN83RmGqw1UKEimJSjo3416Xw&oe=6785EE58&_nc_sid=d885a2',
    'dimensions': {'height': 891, 'width': 1080},
    'gating_info': None,
    'fact_check_overall_rating': None,
    'fact_check_information': None,
    'sensitivity_friction_info': None,
    'sharing_friction_info': {'should_have_sharing_friction': False,
     'bloks_app_url': None},
    'media_overlay_info': None,
    'media_preview': None,
    'display_url': 'https://instagram.fnbo16-1.fna.f

In [ ]:
from datetime import datetime

# Lists to store URLs and corresponding dates
all_urls = []
all_dates = []



# Iterate through each JSON data in the list
for json_data in json_list:
    
    # Extract the list from the 'items' key
    item_list = json_data.get('items', [])
    
    # Iterate through each item in the 'items' list
    for item in item_list:
        
        # Extract the date the item was taken
        taken_date = json_data.get('taken_at_timestamp')
        date_taken = datetime.fromtimestamp(taken_date).strftime("%Y-%m-%d %H:%M:%S")

        # Check if 'carousel_media' is present
        carousel_media = item.get('__typename') == 'XDTGraphSidecar'
        
        # Iterate through each media in the 'carousel_media' list
        for media in carousel_media:
            
            # Extract the image URL from the media
            image_url = media.get('image')
            
            # Check if the image_url field is found inside the 'carousel_media' list

                # Add the image URL and corresponding date to the lists
                
            # Extract the video URL from the media
            
                    # Add the video URL and corresponding date to the lists

        # Handle cases of unique image, instead of carousel
            
            # Add the image URL and corresponding date to the lists

        # Check if 'video_versions' key exists
                

In [ ]:
# Create a directory to store downloaded files

# Create subfolders for images and videos

# Initialize counters for images and videos


# Iterate through URLs in the all_urls list and download media

    # Extract file extension from the URL

    # Determine the file name based on the URL
    
        # Default to the main download directory for other file types

    # Save the file to the appropriate folder

    
    # Write the content of the response to the file
                

# Print a message indicating the number of downloaded files and the download directory